<a href="https://colab.research.google.com/github/ayasherby/content/blob/main/Grad_model_Tooth_segmentation%20almost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Arnold0210/TEETH-RECOGNITION-WITH-MACHINE-LEARNING


Cloning into 'TEETH-RECOGNITION-WITH-MACHINE-LEARNING'...
remote: Enumerating objects: 32505, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 32505 (delta 19), reused 17 (delta 9), pack-reused 32477
Receiving objects: 100% (32505/32505), 6.29 GiB | 27.65 MiB/s, done.
Resolving deltas: 100% (5099/5099), done.
Updating files: 100% (14166/14166), done.


In [2]:
!pip install opencv-python-headless numpy tqdm

In [3]:
!pip install numpy pandas matplotlib scikit-learn

In [4]:
import os

# Change the working directory to your project root
project_root = '/content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/Experiment/Source'
os.chdir(project_root)

# Verify the change
print("Current Working Directory: ", os.getcwd())

Current Working Directory:  /content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/Experiment/Source


In [16]:
!python main.py

PreProcessing Directory Already Exists.
FeatureExtraction Directory Already Exists.
Sampling Directory Already Exists.
Classification Directory Already Exists.
Segmentation Directory Already Exists.
BarPlot Directory Already Exists.
BarPlotCh1 Directory Already Exists.
BarPlotCh2 Directory Already Exists.
BarPlotCh3 Directory Already Exists.
Mask Directory Already Exists.
MaskOverlay Directory Already Exists.
Inverse Directory Already Exists.
ResizeImages Directory Already Exists.
RGB2YCbCr Directory Already Exists.
HSV Directory Already Exists.
LAB Directory Already Exists.
Lab_ Directory Already Exists.
ResizeImages Directory Already Exists.
Segmentation Directory Already Exists.
BarPlot Directory Already Exists.
BarPlotCh1 Directory Already Exists.
BarPlotCh2 Directory Already Exists.
BarPlotCh3 Directory Already Exists.
Mask Directory Already Exists.
MaskOverlay Directory Already Exists.
Inverse Directory Already Exists.
ResizeImages Directory Already Exists.
RGB2YCbCr Directory Al

In [ ]:
#MAIN i think i t will work

import errno
import os
import sys

import cv2 as cv
import numpy as np
from tqdm import tqdm

import Classification as Cl
import FeatureExtraction as fE
import PreProcessingData as pD
import ReadImages as rI


def show(image):
    cv.imshow('Imagen ', image)
    cv.waitKey(0)
    cv.destroyAllWindows()


class MainClass:
    PROJECT_PATH = os.path.join(os.getcwd(), os.path.pardir)
    PATH_IMAGES_ORIGINAL = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "DATASET - Original"))
    PATH_Labels = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "Labels"))
    PATH_LabelsXML = os.path.abspath(os.path.join(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "Labels"), "LabelsXML"))
    PATH_IMAGES = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "DATASET"))
    readimages = None
    preprocessing = None
    featureExtraction = None
    clasification = None

    def __init__(self):
        try:
            if not os.path.exists(self.PROJECT_PATH + 'PreProcessing'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'PreProcessing'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('PreProcessing Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'FeatureExtraction'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'FeatureExtraction'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('FeatureExtraction Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'Sampling'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'Sampling'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('Sampling Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'Classification'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'Classification'))
                print('Directory Classification Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('Classification Directory Already Exists.')
            else:
                raise
        self.readimages = rI.LoadData(self.PATH_IMAGES_ORIGINAL)
        self.preProcessing = pD.PreProcessingData(self.PROJECT_PATH, self.PATH_IMAGES_ORIGINAL)
        self.featureExtraction = fE.FeatureExtraction(self.PROJECT_PATH, self.PATH_IMAGES_ORIGINAL)
        self.clasification = Cl.Classification(self.PROJECT_PATH)

    def main_run(self):
        # Se declaran las clases para poder utilizar los elementos
        read = self.readimages
        pp = self.preProcessing
        fe = self.featureExtraction

        # Se lee el nombre y la imagen que se encuentre en el PATH del dataset ORIGINAL
        # img, name = read.read_One_Image(self.PATH_IMAGES)
        # Se lee el nombre y la imagen que se encuentre en el PATH del dataset RECORTADO
        img, name = read.read_One_Image(self.PATH_IMAGES)

        # Se obtiene las dimensiones de la imagen original
        height_ori, width_ori, depth_ori = img.shape
        # print("Image original shape: \n Height:", height_ori, ", Width:", width_ori)

        # Se realiza un ajuste de tamaño para reducir la imagen a unas dimensiones de 600x400
        img_resize = pp.resize_Image(img, name)

        # La imagen reajustada se convierte de BGR a RGB
        img_resize = cv.cvtColor(img_resize, cv.COLOR_BGR2RGB)
        # Se convierte la imagen de RGB a HSV
        hsv_image = pp.rgb_2_HSV(img_resize, name)
        # Se saca la imagen en pila de tono de rojos
        stack, name = pp.stackColors(hsv_image, name)
        # Se saca los histogramas por imagen para determinar el rango de color de los dientes
        pp.hsv_hist(hsv_image, name)
        #
        # plt.imshow(img_resize)
        # Blur image slightly
        name, blurimage = pp.blurImage(img_resize, name)
        pp.show_mask(blurimage, name)
        pp.overlay_mask(blurimage, img_resize, name)

        # Se obtiene la rueda cromatica de la imágen
        # pp.getChromatiColor(img_resize,name,fe)

        # img_rgb2ycbcr = pp.rgb_2_YCrCb(img_resize, name)
        img_rgb2hsv = pp.rgb_2_HSV(img_resize, name)

    '''easygui.msgbox("Image original shape: \n Height:" + str(height_ori) + "px, Width:" + str(width_ori) + "px" +
                   "\n Image Resize shape: \n Height:" + str(height_res) + "px, Width:" + str(width_res) + "px",
                   image=os.path.join(os.path.join(os.getcwd(), os.path.pardir),
                                      'PreProcessing/ResizeImages/' + name),
                   title="Image Shape - PreProcessing ")'''

    def savebin(self):
        read = self.readimages
        pp = self.preProcessing
        fe = self.featureExtraction
        images, names = read.read_Images(self.PATH_IMAGES_P)
        for image_point, name_point in zip(images, names):
            img_resize = pp.resize_Image(image_point, name_point)
            img_resize = cv.cvtColor(img_resize, cv.COLOR_BGR2RGB)
            img_resize = cv.cvtColor(img_resize, cv.COLOR_RGB2GRAY)
            pp.bin(img_resize, name_point)

    def main_alldataset(self):
        cc = self.clasification

        # Ask for user input for number of iterations and dataset splits
        times_execution = int(input('\nIndicate the number of iterations: '))
        folds = int(input('\nNumber of folds to split the dataset: '))
        test_size = float(input('\nPercentage of test dataset (e.g., 20 for 20%): '))

        for i in range(times_execution):
            # Read features and labels
            filefeaturespath = os.path.join(self.PROJECT_PATH, 'FeatureExtraction', 'features.csv')
            names, features = cc.readfeatures(filefeaturespath)
            labels = cc.readLabels(self.PATH_Labels)

            # Cross-validation
            X, Y = cc.CrossValidation(features, labels, test_size / 100)
            SVM_results, DT_results, KNN_results = cc.classification(
                self.PATH_IMAGES, X, Y, folds, tags=['0', '1', '2', '3'],
                target_names=['a1', 'a2', 'a3', 'a35'],
                vals_to_replace={'a1': '0', 'a2': '1', 'a3': '2', 'a35': '3'}
            )

            print(f'\n--------- Iteration {i+1} ---------')

            for SVM_result, DT_result, KNN_result in zip(SVM_results, DT_results, KNN_results):
                matrix_confusion_SVM, report_clasification_SVM, score_accuracy_SVM = SVM_result
                matrix_confusion_DT, report_clasification_DT, score_accuracy_DT = DT_result
                matrix_confusion_KNN, report_clasification_KNN, score_accuracy_KNN = KNN_result

                print('\n------- SVM Results -------')
                print('Confusion Matrix:\n', matrix_confusion_SVM)
                print('Classification Report:\n', report_clasification_SVM)
                print('Mean Accuracy:', np.mean(score_accuracy_SVM))

                print('\n------- Decision Tree Results -------')
                print('Confusion Matrix:\n', matrix_confusion_DT)
                print('Classification Report:\n', report_clasification_DT)
                print('Mean Accuracy:', np.mean(score_accuracy_DT))

                print('\n------- KNN Results -------')
                print('Confusion Matrix:\n', matrix_confusion_KNN)
                print('Classification Report:\n', report_clasification_KNN)
                print('Mean Accuracy:', np.mean(score_accuracy_KNN))

            print('--------- End of Iteration ---------')

if __name__ == '__main__':
    tesis = MainClass()
    tesis.main_alldataset()
    print('Experiment execution completed.')


In [ ]:
#CLASSIFCATRON CLASS
import errno
import os
from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm
from sklearn.metrics import roc_curve, auc, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


class Classification:
    def __init__(self, PATH_PROJECT):
        self.path_project = os.path.join(PATH_PROJECT, 'Classification')
        try:
            if not os.path.exists(self.path_project + 'SVM'):
                self.path_getColor = os.path.join(self.path_project, 'SVM')
                os.mkdir(self.path_getColor)
                print('SVM Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('SVM Directory Already Exists.')
            else:
                raise

    def readfeatures(self, features_Path):
        # featuresFile = open(features_Path, "r")
        # featuresData = csv.reader(featuresFile)
        featuresFile = pd.read_csv(features_Path, sep=',', header=None)
        names = featuresFile.iloc[:, 0]
        features = featuresFile.iloc[:, 1:]
        shapefile = featuresFile.shape
        col = []
        for x in range(0, shapefile[1]):
            if x == 0:
                col.append("NAME")
            else:
                col.append("VALOR-" + str(x))
        featuresFile.columns = col
        # print(featuresFile)
        return names, features

    def readLabels(self, labels_path):
        labels_path = os.path.join(labels_path, 'Labels.csv')
        labels = pd.read_csv(labels_path, sep=',', header=[0])
        return labels

    def classificatorSVM(self, features, labels):
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        clf = svm.SVC(gamma='scale')
        clf.fit(X, tags)
        return clf

    def DecisionTree(self, features, labels):
        dt = DecisionTreeClassifier(random_state=30, max_depth=300)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        dt.fit(X, tags)

        return dt

    def KNN(self, features, labels):
        knn = KNeighborsClassifier(n_neighbors=155, algorithm='auto', weights='distance', n_jobs=-1)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        knn.fit(X, tags)
        return knn

    def classification(self, path_dataset, features, labels, n_splits, tags, target_names, vals_to_replace):
        pd.options.mode.chained_assignment = None
        onlyfiles = [f for f in listdir(path_dataset) if
                     isfile(join(path_dataset, f))]
        #print(onlyfiles)
        k_folds = KFold(n_splits=n_splits)
        SVM = []
        KNN = []
        DT = []
        for stage, feature in zip(labels, features):
            stage['Color'] = stage['Color'].map(vals_to_replace)
            labels_color = stage['Color'].to_numpy().tolist()
            images_name = stage['Nombre de la imagen'].to_numpy().tolist()
            svm_training_Score = []
            confusion_matrix_svm = []
            confusion_matrix_dt = []
            confusion_matrix_knn = []
            classification_report_svm = []
            classification_report_dt = []
            classification_report_knn = []
            score_accuracy_SVM = []
            score_accuracy_DT = []
            score_accuracy_KNN = []
            #index_images_name = [onlyfiles.index(element) for element in images_name if element in onlyfiles]
            index_images_name = [images_name.index(file.split('.')[0]) for file in onlyfiles if file.split('.')[0] in images_name]
            #index_images_name = []
            k_folds.get_n_splits(index_images_name)
            #print(list(images_name.index))

            for train_index, test_index in k_folds.split(index_images_name):
                train_label = []
                test_label = []
                train_features = []
                test_features = []
                print("Training indices:", [index_images_name[i] for i in train_index])
                print("Testing indices:", [index_images_name[i] for i in test_index])

                for i in train_index:

                    currentFilename=images_name[i]
                    train_features.append(feature.to_numpy().tolist()[index_images_name[i]])
                    # train_features.append(feature.to_numpy()[images_name.str(onlyfiles[i].split('.')[0])])
                    train_label.append(labels_color[index_images_name[i]])

                SVM_Classifier = self.classificatorSVM(train_features, train_label)
                DT_Classifier = self.DecisionTree(train_features, train_label)
                KNN_Classifier = self.KNN(train_features, train_label)

                for i in test_index:

                    test_features.append(feature.to_numpy()[index_images_name[i]])
                    test_label.append(labels_color[index_images_name[i]])
                predict_label_SVM = SVM_Classifier.predict(test_features)
                predict_label_DT = DT_Classifier.predict(test_features)
                predict_label_KNN = KNN_Classifier.predict(test_features)

                confusionMatrixSVM = confusion_matrix(test_label, predict_label_SVM)
                confusionMatrixDT = confusion_matrix(test_label, predict_label_DT)
                confusionMatrixKNN = confusion_matrix(test_label, predict_label_KNN)

                classification_report_svm.append(
                    classification_report(test_label, predict_label_SVM, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_dt.append(
                    classification_report(test_label, predict_label_DT, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_knn.append(
                    classification_report(test_label, predict_label_KNN, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))

                confusion_matrix_svm.append(confusionMatrixSVM)
                confusion_matrix_dt.append(confusionMatrixDT)
                confusion_matrix_knn.append(confusionMatrixKNN)

                score_accuracy_SVM.append(accuracy_score(test_label, predict_label_SVM))
                score_accuracy_DT.append(accuracy_score(test_label, predict_label_DT))
                score_accuracy_KNN.append(accuracy_score(test_label, predict_label_KNN))
            SVM_RESULTS = [confusion_matrix_svm, classification_report_svm, score_accuracy_SVM]
            DT_RESULTS = [confusion_matrix_dt, classification_report_dt, score_accuracy_DT]
            KNN_RESULTS = [confusion_matrix_knn, classification_report_knn, score_accuracy_KNN]
            SVM.append(SVM_RESULTS)
            KNN.append(KNN_RESULTS)
            DT.append(DT_RESULTS)
        return SVM, KNN, DT

    def CrossValidation(self, image, labels, test_size):
        X = []
        Y = []
        X_train, X_test, y_train, y_test = train_test_split(image, labels, test_size=test_size)
        X.append(X_train)
        X.append(X_test)
        Y.append(y_train)
        Y.append(y_test)
        print()
        return X, Y

    def ROC_CURVE(self, label_test, label_score):
        n_classes = label_test.shape
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(label_test[:, i], label_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(label_test.ravel(), label_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # Plot of a ROC curve for a specific class
        plt.figure()
        plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr["micro"], tpr["micro"],
                 label='micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]))
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                           ''.format(i, roc_auc[i]))

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Some extension of Receiver operating characteristic to multi-class')
        plt.legend(loc="lower right")
        plt.show()

In [ ]:
#this just to know how to solve the Value ERrro in a simple version
from sklearn.model_selection import KFold
# Define the whole dataset
AllData = ["V","X", "Y", "T", "R", "A", "O", "Q", "N", "U", "G","P"]

# Define the training dataset (70% of AllData)
porData = ["Y", "T", "R", "A", "O", "Q", "N", "U"]

# Get the indices of the training dataset in the whole dataset
porDataIndex = [AllData.index(element) for element in porData if element in AllData]
print("AllData:",AllData)
print("portData :",porData)
print("indes all:",porDataIndex)

# Define the K-fold object with K=5
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Split the data into training and testing sets
for train_index, test_index in kf.split(porDataIndex):
    train_features=[]
    test_features=[]

    print("Training indices:", [porDataIndex[i] for i in train_index])
    print("Testing indices:", [porDataIndex[i] for i in test_index])

    for i in train_index:
        current=porData[i]
        train_features.append(porDataIndex[i])
        print(f"image with index ={porDataIndex[i]} and name {current} is added to training")

    for i in test_index:
        current=porData[i]
        test_features.append(porDataIndex[i])
        print(f"image with index ={porDataIndex[i]} and name {current} is added to test")

In [ ]:
#important CLASSIFICATION CLOSE FROM
import errno
import os
from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm
from sklearn.metrics import roc_curve, auc, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


class Classification:
    def __init__(self, PATH_PROJECT):
        self.path_project = os.path.join(PATH_PROJECT, 'Classification')
        try:
            if not os.path.exists(self.path_project + 'SVM'):
                self.path_getColor = os.path.join(self.path_project, 'SVM')
                os.mkdir(self.path_getColor)
                print('SVM Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('SVM Directory Already Exists.')
            else:
                raise

    def readfeatures(self, features_Path):
        # featuresFile = open(features_Path, "r")
        # featuresData = csv.reader(featuresFile)
        featuresFile = pd.read_csv(features_Path, sep=',', header=None)
        names = featuresFile.iloc[:, 0]
        features = featuresFile.iloc[:, 1:]
        shapefile = featuresFile.shape
        col = []
        for x in range(0, shapefile[1]):
            if x == 0:
                col.append("NAME")
            else:
                col.append("VALOR-" + str(x))
        featuresFile.columns = col
        # print(featuresFile)
        return names, features

    def readLabels(self, labels_path):
        labels_path = os.path.join(labels_path, 'Labels.csv')
        labels = pd.read_csv(labels_path, sep=',', header=[0])
        return labels

    def classificatorSVM(self, features, labels):
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        clf = svm.SVC(gamma='scale')
        clf.fit(X, tags)
        return clf

    def DecisionTree(self, features, labels):
        dt = DecisionTreeClassifier(random_state=30, max_depth=300)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        dt.fit(X, tags)

        return dt

    def KNN(self, features, labels):
        knn = KNeighborsClassifier(n_neighbors=200, algorithm='auto', weights='distance', n_jobs=-1)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        knn.fit(X, tags)
        return knn

    def classification(self, path_dataset, features, labels, n_splits, tags, target_names, vals_to_replace):
        pd.options.mode.chained_assignment = None
        onlyfiles = [f for f in listdir(path_dataset) if
                     isfile(join(path_dataset, f))]
        #print(onlyfiles)
        k_folds = KFold(n_splits=n_splits)
        SVM = []
        KNN = []
        DT = []
        for stage, feature in zip(labels, features):
            stage['Color'] = stage['Color'].map(vals_to_replace)
            labels_color = stage['Color'].to_numpy().tolist()
            images_name = stage['Nombre de la imagen'].to_numpy().tolist()
            svm_training_Score = []
            confusion_matrix_svm = []
            confusion_matrix_dt = []
            confusion_matrix_knn = []
            classification_report_svm = []
            classification_report_dt = []
            classification_report_knn = []
            score_accuracy_SVM = []
            score_accuracy_DT = []
            score_accuracy_KNN = []
            index_images_name=[]
            # Assuming you have corrected the issue with index_images_name
            # Generate index_images_name based on matching base names (without extensions)
            index_images_name = [images_name.index(file.split('.')[0]) for file in onlyfiles if file.split('.')[0] in images_name]            # Debugging the index calculation

            # Verify the correctness of index_images_name
            print(f"\nLength of index_images_name: {len(index_images_name)}",index_images_name[0],index_images_name[-1],"\n")

            for train_index, test_index in k_folds.split(index_images_name):
                train_label = []
                test_label = []
                train_features = []
                test_features = []
                print(f"\n the train_index {len(train_index)}=",train_index ,"\n")
                print(f"\n the test_index {len(test_index)}=",test_index ,"\n")

                for i in train_index:
                    train_features.append(feature.to_numpy().tolist()[images_name.index(onlyfiles[i].split('.')[0])])
                    # train_features.append(feature.to_numpy()[images_name.str(onlyfiles[i].split('.')[0])])
                    train_label.append(labels_color[images_name.index(onlyfiles[i].split('.')[0])])
                SVM_Classifier = self.classificatorSVM(train_features, train_label)
                DT_Classifier = self.DecisionTree(train_features, train_label)
                KNN_Classifier = self.KNN(train_features, train_label)

                for i in test_index:
                    test_features.append(feature.to_numpy()[images_name.index(str(onlyfiles[i].split('.')[0]))])
                    test_label.append(labels_color[images_name.index(str(onlyfiles[i].split('.')[0]))])
                predict_label_SVM = SVM_Classifier.predict(test_features)
                predict_label_DT = DT_Classifier.predict(test_features)
                predict_label_KNN = KNN_Classifier.predict(test_features)

                confusionMatrixSVM = confusion_matrix(test_label, predict_label_SVM)
                confusionMatrixDT = confusion_matrix(test_label, predict_label_DT)
                confusionMatrixKNN = confusion_matrix(test_label, predict_label_KNN)

                classification_report_svm.append(
                    classification_report(test_label, predict_label_SVM, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_dt.append(
                    classification_report(test_label, predict_label_DT, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_knn.append(
                    classification_report(test_label, predict_label_KNN, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))

                confusion_matrix_svm.append(confusionMatrixSVM)
                confusion_matrix_dt.append(confusionMatrixDT)
                confusion_matrix_knn.append(confusionMatrixKNN)

                score_accuracy_SVM.append(accuracy_score(test_label, predict_label_SVM))
                score_accuracy_DT.append(accuracy_score(test_label, predict_label_DT))
                score_accuracy_KNN.append(accuracy_score(test_label, predict_label_KNN))
            SVM_RESULTS = [confusion_matrix_svm, classification_report_svm, score_accuracy_SVM]
            DT_RESULTS = [confusion_matrix_dt, classification_report_dt, score_accuracy_DT]
            KNN_RESULTS = [confusion_matrix_knn, classification_report_knn, score_accuracy_KNN]
            SVM.append(SVM_RESULTS)
            KNN.append(KNN_RESULTS)
            DT.append(DT_RESULTS)
        return SVM, KNN, DT

    def CrossValidation(self, image, labels, test_size):
        X = []
        Y = []
        X_train, X_test, y_train, y_test = train_test_split(image, labels, test_size=test_size)
        X.append(X_train)
        X.append(X_test)
        Y.append(y_train)
        Y.append(y_test)
        print()
        return X, Y

    def ROC_CURVE(self, label_test, label_score):
        n_classes = label_test.shape
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(label_test[:, i], label_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(label_test.ravel(), label_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # Plot of a ROC curve for a specific class
        plt.figure()
        plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr["micro"], tpr["micro"],
                 label='micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]))
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                           ''.format(i, roc_auc[i]))

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Some extension of Receiver operating characteristic to multi-class')
        plt.legend(loc="lower right")
        plt.show()


PreProcessing Directory Already Exists.
FeatureExtraction Directory Already Exists.
Sampling Directory Already Exists.
Classification Directory Already Exists.
Segmentation Directory Already Exists.
BarPlot Directory Already Exists.
BarPlotCh1 Directory Already Exists.
BarPlotCh2 Directory Already Exists.
BarPlotCh3 Directory Already Exists.
Mask Directory Already Exists.
MaskOverlay Directory Already Exists.
Inverse Directory Already Exists.
ResizeImages Directory Already Exists.
RGB2YCbCr Directory Already Exists.
HSV Directory Already Exists.
LAB Directory Already Exists.
Lab_ Directory Already Exists.
ResizeImages Directory Already Exists.
Segmentation Directory Already Exists.
BarPlot Directory Already Exists.
BarPlotCh1 Directory Already Exists.
BarPlotCh2 Directory Already Exists.
BarPlotCh3 Directory Already Exists.
Mask Directory Already Exists.
MaskOverlay Directory Already Exists.
Inverse Directory Already Exists.
ResizeImages Directory Already Exists.
RGB2YCbCr Directory Al

  0%|          | 0/5 [00:00<?, ? times/s]


Cantidad de folios a seperarar el conjunto de datos:5

 Porcentaje de division del conjunto de datos trianing/test:20


  0%|          | 0/5 [00:02<?, ? times/s]

ValueError: '101_0138' is not in list

# **`Saved codes to run`**

In [ ]:
#  Copyright (c) 2019. Arnold Julian Herrera Quiñones -  Cristhian Camilo Arce García.
#  All Rights Reserved
#
#  This product is protected by copyright and distributed under
#  licenses restricting copying, distribution, and decompilation.
#  It is forbidden the use partial or global of this algorithm  unless authors written permission.
#
import errno
import os
from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm
from sklearn.metrics import roc_curve, auc, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


class Classification:
    def __init__(self, PATH_PROJECT):
        self.path_project = os.path.join(PATH_PROJECT, 'Classification')
        try:
            if not os.path.exists(self.path_project + 'SVM'):
                self.path_getColor = os.path.join(self.path_project, 'SVM')
                os.mkdir(self.path_getColor)
                print('SVM Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('SVM Directory Already Exists.')
            else:
                raise

    def readfeatures(self, features_Path):

        featuresFile = pd.read_csv(features_Path, sep=',', header=None)
        names = featuresFile.iloc[:, 0]
        features = featuresFile.iloc[:, 1:]
        shapefile = featuresFile.shape
        col = []
        for x in range(0, shapefile[1]):
            if x == 0:
                col.append("NAME")
            else:
                col.append("VALOR-" + str(x))
        featuresFile.columns = col
        # print(featuresFile)
        return names, features

    def readLabels(self, labels_path):
        labels_path = os.path.join(labels_path, 'Labels.csv')
        labels = pd.read_csv(labels_path, sep=',', header=[0])
        return labels

    def classificatorSVM(self, features, labels):
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        clf = svm.SVC(gamma='scale')
        clf.fit(X, tags)
        return clf

    def DecisionTree(self, features, labels):
        dt = DecisionTreeClassifier(random_state=30, max_depth=300)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        dt.fit(X, tags)

        return dt

    def KNN(self, features, labels):
        knn = KNeighborsClassifier(n_neighbors=200, algorithm='auto', weights='distance', n_jobs=-1)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        knn.fit(X, tags)
        return knn

    def classification(self, path_dataset, features, labels, n_splits, tags, target_names, vals_to_replace):
        pd.options.mode.chained_assignment = None
        onlyfiles = [f for f in listdir(path_dataset) if
                     isfile(join(path_dataset, f))]
        #print(onlyfiles)
        k_folds = KFold(n_splits=n_splits)
        SVM = []
        KNN = []
        DT = []
        for stage, feature in zip(labels, features):
            stage['Color'] = stage['Color'].map(vals_to_replace)
            labels_color = stage['Color'].to_numpy().tolist()
            images_name = stage['Nombre de la imagen'].to_numpy().tolist()
            svm_training_Score = []
            confusion_matrix_svm = []
            confusion_matrix_dt = []
            confusion_matrix_knn = []
            classification_report_svm = []
            classification_report_dt = []
            classification_report_knn = []
            score_accuracy_SVM = []
            score_accuracy_DT = []
            score_accuracy_KNN = []
            index_images_name = []
            k_folds.get_n_splits(index_images_name)
            #print(list(images_name.index))

            """
            1- first thing we need to make a index_images_name list that retreveis the indeces of the image_name list
            for example it will takes from 0 to 777 index this its the range of the image_name that included
            2- for the for loop when it spilts the k_fold it will spilt the train_index and test_index based on the
            list of the index_images_name which contains 0--->777 this means that it takes from the indexcess for example
            train_index =(0-->621) and tes_index =(622-->777) this way we guraannte that it takes from the image_name and
            now for the for loop of the train_index let's suppose that i = 160 and when we retreves its name from onlyfiles
            it gives for example PP.JPG after the spilt('.') the current_filename = "PP" and then it will go into if condition
            which asks is this image: " PP " included in the list image_name(list of names) ?
            so it supposed to return True because the index of this image is already taken from index_images_name which are
            the list of indices of the image_name itself so it should find this image and then in the appeanding to train_featues
            we could use the image_name.index or retireves its value from i
            """
            for train_index, test_index in k_folds.split(index_images_name):
                train_label = []
                test_label = []
                train_features = []
                test_features = []
                # Assuming `onlyfiles` contains all filenames in the dataset
                # `images_name` contains filenames corresponding to the training dataset

                for i in train_index:
                    if i < len(images_name):  # Ensure i corresponds to a valid index in `images_name`
                        current_filename = onlyfiles[i].split('.')[0].strip()
                        if current_filename in images_name:  # Check if filename is part of the training dataset
                            # Process the current_filename as part of the training dataset
                            train_features.append(feature.to_numpy()[images_name.index(str(current_filename))])
                            train_label.append(labels_color[images_name.index(str(current_filename))])
                        else:
                            print(f"Warning: '{current_filename}' not found in training dataset.")
                    else:
                        print(f"Warning: Index {i} out of range for training dataset.")

                SVM_Classifier = self.classificatorSVM(train_features, train_label)
                DT_Classifier = self.DecisionTree(train_features, train_label)
                KNN_Classifier = self.KNN(train_features, train_label)
                for i in test_index:
                    if i < len(onlyfiles):  # Ensure `i` corresponds to a valid index in `onlyfiles`
                        current_filename = onlyfiles[i].split('.')[0].strip()
                        if current_filename in images_name:  # Check if filename is part of the training dataset
                            # Process the `current_filename` as part of the test dataset
                            test_features.append(feature.to_numpy()[images_name.index(str(current_filename))])
                            test_label.append(labels_color[images_name.index(str(current_filename))])
                        else:
                            print(f"Warning: '{current_filename}' not found in training dataset.")
                    else:
                        print(f"Warning: Index {i} out of range for `onlyfiles` dataset.")


                predict_label_SVM = SVM_Classifier.predict(test_features)
                predict_label_DT = DT_Classifier.predict(test_features)
                predict_label_KNN = KNN_Classifier.predict(test_features)

                confusionMatrixSVM = confusion_matrix(test_label, predict_label_SVM)
                confusionMatrixDT = confusion_matrix(test_label, predict_label_DT)
                confusionMatrixKNN = confusion_matrix(test_label, predict_label_KNN)

                classification_report_svm.append(
                    classification_report(test_label, predict_label_SVM, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_dt.append(
                    classification_report(test_label, predict_label_DT, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_knn.append(
                    classification_report(test_label, predict_label_KNN, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))

                confusion_matrix_svm.append(confusionMatrixSVM)
                confusion_matrix_dt.append(confusionMatrixDT)
                confusion_matrix_knn.append(confusionMatrixKNN)

                score_accuracy_SVM.append(accuracy_score(test_label, predict_label_SVM))
                score_accuracy_DT.append(accuracy_score(test_label, predict_label_DT))
                score_accuracy_KNN.append(accuracy_score(test_label, predict_label_KNN))
            SVM_RESULTS = [confusion_matrix_svm, classification_report_svm, score_accuracy_SVM]
            DT_RESULTS = [confusion_matrix_dt, classification_report_dt, score_accuracy_DT]
            KNN_RESULTS = [confusion_matrix_knn, classification_report_knn, score_accuracy_KNN]
            SVM.append(SVM_RESULTS)
            KNN.append(KNN_RESULTS)
            DT.append(DT_RESULTS)
        return SVM, KNN, DT

    def CrossValidation(self, image, labels, test_size):
        X = []
        Y = []
        X_train, X_test, y_train, y_test = train_test_split(image, labels, test_size=test_size)
        X.append(X_train)
        X.append(X_test)
        Y.append(y_train)
        Y.append(y_test)
        print()
        return X, Y

    def ROC_CURVE(self, label_test, label_score):
        n_classes = label_test.shape
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(label_test[:, i], label_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(label_test.ravel(), label_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # Plot of a ROC curve for a specific class
        plt.figure()
        plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr["micro"], tpr["micro"],
                 label='micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]))
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                           ''.format(i, roc_auc[i]))

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Some extension of Receiver operating characteristic to multi-class')
        plt.legend(loc="lower right")
        plt.show()


In [ ]:
#CLASSIFICATION ME
import errno
import os
from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm
from sklearn.metrics import roc_curve, auc, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


class Classification:
    def __init__(self, PATH_PROJECT):
        self.path_project = os.path.join(PATH_PROJECT, 'Classification')
        try:
            if not os.path.exists(self.path_project + 'SVM'):
                self.path_getColor = os.path.join(self.path_project, 'SVM')
                os.mkdir(self.path_getColor)
                print('SVM Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('SVM Directory Already Exists.')
            else:
                raise

    def readfeatures(self, features_Path):
        # featuresFile = open(features_Path, "r")
        # featuresData = csv.reader(featuresFile)
        featuresFile = pd.read_csv(features_Path, sep=',', header=None)
        names = featuresFile.iloc[:, 0]
        features = featuresFile.iloc[:, 1:]
        shapefile = featuresFile.shape
        col = []
        for x in range(0, shapefile[1]):
            if x == 0:
                col.append("NAME")
            else:
                col.append("VALOR-" + str(x))
        featuresFile.columns = col
        # print(featuresFile)
        print("\nNumber of features :",len(features),"\n")
        print("\nNumber of names :",len(names),"\n")
        return names, features

    def readLabels(self, labels_path):
        labels_path = os.path.join(labels_path, 'Labels.csv')
        labels = pd.read_csv(labels_path, sep=',', header=[0])
        print("\nNumber of labels :",len(labels),"\n")
        return labels


    def classificatorSVM(self, features, labels):
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        clf = svm.SVC(gamma='scale')
        clf.fit(X, tags)
        return clf

    def DecisionTree(self, features, labels):
        dt = DecisionTreeClassifier(random_state=30, max_depth=300)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        dt.fit(X, tags)

        return dt

    def KNN(self, features, labels):
        knn = KNeighborsClassifier(n_neighbors=200, algorithm='auto', weights='distance', n_jobs=-1)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        knn.fit(X, tags)
        return knn

    def classification(self, path_dataset, features, labels, n_splits, tags, target_names, vals_to_replace):
        pd.options.mode.chained_assignment = None
        num_images = len(os.listdir(path_dataset))
        print(f"Number of images in dataset Classi def: {num_images}")
        onlyfiles = [f for f in listdir(path_dataset) if
                     isfile(join(path_dataset, f))] # all dataset=973
        onlyfiles = sorted(onlyfiles)
        print("\nNumber of only files :",len(onlyfiles),"\n")

        print("\nNumber of features in classi def :",len(features),"\n")
        print("\nNumber of labels classi def :",len(labels),"\n")
        k_folds = KFold(n_splits=n_splits)
        SVM = []
        KNN = []
        DT = []
        for stage, feature in zip(labels, features):
            stage['Color'] = stage['Color'].map(vals_to_replace)
            labels_color = stage['Color'].to_numpy().tolist()
            images_name = stage['Nombre de la imagen'].to_numpy().tolist()
            print(f"Sample of image names in current stage {len(images_name)}:")
            #print("Images names from labels.csv:", images_name)
            print("\nNumber of training labels_color in  labels.csv :",len(labels_color),"\n")
            svm_training_Score = []
            confusion_matrix_svm = []
            confusion_matrix_dt = []
            confusion_matrix_knn = []
            classification_report_svm = []
            classification_report_dt = []
            classification_report_knn = []
            score_accuracy_SVM = []
            score_accuracy_DT = []
            score_accuracy_KNN = []
            index_images_name = []
            k_folds.get_n_splits(index_images_name)
            #print(list(images_name.index))
            imageIsFound=0
            missing_images_count = 0
            print("\nonlyfile :",onlyfiles[:5])
            print("\n Image_name :",images_name[:5])

            #missing_images_list = []
            for train_index, test_index in k_folds.split(images_name):
                print(f"Train Index: {len(train_index)}")
                print(f"Test Index: {len(test_index)}")
                print("\hello i'm inside for KFOLD\n")
                train_label = []
                test_label = []
                train_features = []
                test_features = []

                # Assuming `features` is a list of image names or identifiers
                print("\nSample of test features",test_features,"\n")
                print("\nSample of train features",train_features,"\n")
                print("\nSample of train label",train_label,"\n")
                print("Sample of test label",test_label,"\n")
                print("\nSample of  train_index",train_index,"\n")
                print("Sample of test index",test_index,"\n")



                for i in train_index:

                  current_filename = onlyfiles[i].split('.')[0].strip()  # Get the filename without extension and strip whitespace

                  if current_filename in images_name:
                      train_features.append(feature.to_numpy().tolist()[images_name.index(current_filename)])
                      #print(f"\nhello I'm inside For LOOP for train index of image name: {current_filename}\n")
                      #print(f"\nhello I'm inside For loop of train feature is added to {current_filename}\n")
                      train_label.append(labels_color[images_name.index(current_filename)])
                      #print(f"\nhello I'm inside For loop of train label is added to : {current_filename}\n")
                      imageIsFound += 1
                  else:

                      print(f"Warning: '{current_filename}' not found in images_name because its index of {i}. Skipping this image.")
                      missing_images_count +=1

                print(f"Number of found images: {imageIsFound}")
                print(f"Number of missing images: {missing_images_count}")
                print(f"Total number of images: {missing_images_count+imageIsFound}\n")
                print(f"Total number of Train_features : {len(train_features)}")
                print(f"Total number of train_label : {len(train_label)}")
                SVM_Classifier = self.classificatorSVM(train_features, train_label)
                DT_Classifier = self.DecisionTree(train_features, train_label)
                KNN_Classifier = self.KNN(train_features, train_label)
                print(f"\nhello I'm inside For loop of Train index is **END** to ALL classifiers \n")

                for i in test_index:
                            #print(f"\nhello I'm inside For LOOP for Test index of image name: {current_filename}\n")
                            test_features.append(feature.to_numpy()[images_name.index(str(onlyfiles[i].split('.')[0]))])
                            #test_features.append(feature.to_numpy().tolist()[images_name.index(current_filename)])
                            print(f"\nhello I'm inside For loop of Test  feature is added to {current_filename}\n")
                            test_label.append(labels_color[images_name.index(str(onlyfiles[i].split('.')[0]))])
                            print(f"\nhello I'm inside For loop of Test Label is added to {current_filename}\n")


                predict_label_SVM = SVM_Classifier.predict(test_features)
                predict_label_DT = DT_Classifier.predict(test_features)
                predict_label_KNN = KNN_Classifier.predict(test_features)
                print(f"\n I'm inside For loop of test  index is **END** to ALL classifiers of image : {current_filename}\n")

                confusionMatrixSVM = confusion_matrix(test_label, predict_label_SVM)
                confusionMatrixDT = confusion_matrix(test_label, predict_label_DT)
                confusionMatrixKNN = confusion_matrix(test_label, predict_label_KNN)
                print(f"\n I'm inside For loop of Confusion matrix is **ADDED ** to ALL martiecs of image : {current_filename}\n")

                classification_report_svm.append(
                    classification_report(test_label, predict_label_SVM, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_dt.append(
                    classification_report(test_label, predict_label_DT, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_knn.append(
                    classification_report(test_label, predict_label_KNN, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))

                confusion_matrix_svm.append(confusionMatrixSVM)
                confusion_matrix_dt.append(confusionMatrixDT)
                confusion_matrix_knn.append(confusionMatrixKNN)

                score_accuracy_SVM.append(accuracy_score(test_label, predict_label_SVM))
                score_accuracy_DT.append(accuracy_score(test_label, predict_label_DT))
                score_accuracy_KNN.append(accuracy_score(test_label, predict_label_KNN))
            SVM_RESULTS = [confusion_matrix_svm, classification_report_svm, score_accuracy_SVM]
            DT_RESULTS = [confusion_matrix_dt, classification_report_dt, score_accuracy_DT]
            KNN_RESULTS = [confusion_matrix_knn, classification_report_knn, score_accuracy_KNN]
            SVM.append(SVM_RESULTS)
            KNN.append(KNN_RESULTS)
            DT.append(DT_RESULTS)
        return SVM, KNN, DT

    def CrossValidation(self, image, labels, test_size):
        X = []
        Y = []
        X_train, X_test, y_train, y_test = train_test_split(image, labels, test_size=test_size)
        X.append(X_train)
        X.append(X_test)
        Y.append(y_train)
        Y.append(y_test)
        print()
        return X, Y

    def ROC_CURVE(self, label_test, label_score):
        n_classes = label_test.shape
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(label_test[:, i], label_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(label_test.ravel(), label_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # Plot of a ROC curve for a specific class
        plt.figure()
        plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr["micro"], tpr["micro"],
                 label='micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]))
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                           ''.format(i, roc_auc[i]))

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Some extension of Receiver operating characteristic to multi-class')
        plt.legend(loc="lower right")
        plt.show()


In [ ]:
#CLASSIFICATION ME reshape
import errno
import os
from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.metrics import roc_curve, auc, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


class Classification:
    def __init__(self, PATH_PROJECT):
        self.path_project = os.path.join(PATH_PROJECT, 'Classification')
        try:
            if not os.path.exists(self.path_project + 'SVM'):
                self.path_getColor = os.path.join(self.path_project, 'SVM')
                os.mkdir(self.path_getColor)
                print('SVM Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('SVM Directory Already Exists.')
            else:
                raise

    def readfeatures(self, features_Path):
        # featuresFile = open(features_Path, "r")
        # featuresData = csv.reader(featuresFile)
        featuresFile = pd.read_csv(features_Path, sep=',', header=None)
        names = featuresFile.iloc[:, 0]
        features = featuresFile.iloc[:, 1:]
        shapefile = featuresFile.shape
        col = []
        for x in range(0, shapefile[1]):
            if x == 0:
                col.append("NAME")
            else:
                col.append("VALOR-" + str(x))
        featuresFile.columns = col
        # print(featuresFile)
        print("\nNumber of features :",len(features),"\n")
        print("\nNumber of names :",len(names),"\n")
        return names, features

    def readLabels(self, labels_path):
        labels_path = os.path.join(labels_path, 'Labels.csv')
        labels = pd.read_csv(labels_path, sep=',', header=[0])
        print("\nNumber of labels :",len(labels),"\n")
        return labels


    def classificatorSVM(self, features, labels):
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        clf = svm.SVC(gamma='scale')
        clf.fit(X, tags)
        return clf

    def DecisionTree(self, features, labels):
        dt = DecisionTreeClassifier(random_state=30, max_depth=300)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        dt.fit(X, tags)

        return dt

    def KNN(self, features, labels):
        knn = KNeighborsClassifier(n_neighbors=200, algorithm='auto', weights='distance', n_jobs=-1)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        knn.fit(X, tags)
        return knn

    def classification(self, path_dataset, features, labels, n_splits, tags, target_names, vals_to_replace):
        pd.options.mode.chained_assignment = None
        num_images = len(os.listdir(path_dataset))
        print(f"Number of images in dataset Classi def: {num_images}")
        onlyfiles = [f for f in listdir(path_dataset) if
                     isfile(join(path_dataset, f))] # all dataset=973
        print("\nNumber of only files :",len(onlyfiles),"\n")

        print("\nNumber of features in classi def :",len(features),"\n")
        print("\nNumber of labels classi def :",len(labels),"\n")
        k_folds = KFold(n_splits=n_splits)
        SVM = []
        KNN = []
        DT = []
        for stage, feature in zip(labels, features):
            stage['Color'] = stage['Color'].map(vals_to_replace)
            labels_color = stage['Color'].to_numpy().tolist()
            images_name = stage['Nombre de la imagen'].to_numpy().tolist()
            print(f"Sample of image names in current stage {len(images_name)}:")
            #print("Images names from labels.csv:", images_name)
            print("\nNumber of training labels_color in  labels.csv :",len(labels_color),"\n")
            trainingDataset=images_name
            #testDataset = list(set(onlyfiles) - set(images_name))
            #print(f"\n training dataset= {len(trainingDataset)} and the test dataset ={len(testDataset)}\n")\


            svm_training_Score = []
            confusion_matrix_svm = []
            confusion_matrix_dt = []
            confusion_matrix_knn = []
            classification_report_svm = []
            classification_report_dt = []
            classification_report_knn = []
            score_accuracy_SVM = []
            score_accuracy_DT = []
            score_accuracy_KNN = []
            index_images_name = []
            k_folds.get_n_splits(index_images_name)
            #print(list(images_name.index))
            imageIsFoundTrain=0
            imageIsFoundTest=0
            missingTRain_images_count = 0
            missingTest_images_count = 0

            print("\nonlyfile :",onlyfiles[:5])
            print("\n Image_name :",images_name[:5])
            fold_num=0
            #missing_images_list = []
            for train_index, test_index in k_folds.split(trainingDataset):
                print(f"Train Index: {len(train_index)}")
                print(f"Test Index: {len(test_index)}")
                fold_num+=1
                print(f"\n hello i'm inside Fold number = {fold_num}\n")
                train_label = []
                test_label = []
                train_features = []
                test_features = []

                # Assuming `features` is a list of image names or identifiers
                """print("\nSample of test features",test_features,"\n")
                print("\nSample of train features",train_features,"\n")
                print("\nSample of train label",train_label,"\n")
                print("Sample of test label",test_label,"\n")"""
                #print("\nSample of  train_index",train_index,"\n")
                #print("Sample of test index",test_index,"\n")

                for i in train_index:

                      current_filename = onlyfiles[i].split('.')[0].strip()  # Get the filename without extension and strip whitespace

                      if current_filename in trainingDataset:
                          train_features.append(feature.to_numpy().tolist()[images_name.index(current_filename)])
                          #print(f"\nhello I'm inside For LOOP for train index of image name: {current_filename}\n")
                          #print(f"\nhello I'm inside For loop of train feature is added to {current_filename}\n")
                          train_label.append(labels_color[images_name.index(current_filename)])
                          #print(f"\nhello I'm inside For loop of train label is added to : {current_filename}\n")
                          imageIsFoundTrain += 1
                      else:

                          #print(f"Warning: '{current_filename}' not found in images_name because its index of {i}.train.")
                          missingTRain_images_count +=1

                print(f"Number of found images train: {imageIsFoundTrain}")
                print(f"Number of missing images in train: {missingTRain_images_count}")
                print(f"Total number of images train: {missingTRain_images_count+imageIsFoundTrain}\n")
                print(f"Total number of Train_features : {len(train_features)}")
                print(f"Total number of train_label : {len(train_label)}")
                SVM_Classifier = self.classificatorSVM(train_features, train_label)
                DT_Classifier = self.DecisionTree(train_features, train_label)
                KNN_Classifier = self.KNN(train_features, train_label)
                print("\n **************** END Train_index ******************\n")

                for i in test_index:
                    if current_filename in trainingDataset:
                              #print(f"\nhello I'm inside For LOOP for Test index of image name: {current_filename}\n")
                              test_features.append(feature.to_numpy()[images_name.index(str(current_filename))])
                              #test_features.append(feature.to_numpy().tolist()[images_name.index(current_filename)])
                              #print(f"\nhello I'm inside For loop of Test  feature is added to {current_filename}\n")
                              test_label.append(labels_color[images_name.index(str(current_filename))])
                              #print(f"\nhello I'm inside For loop of Test Label is added to {current_filename}\n")
                              imageIsFoundTest +=1

                    else:
                        #print(f"Warning: '{current_filename}' not found in images_name because its index of {i}. in test.")
                        missingTest_images_count +=1


                print(f"Number of found images in test: {imageIsFoundTest}")
                print(f"Number of missing images in test: {missingTest_images_count}")
                print(f"Total number of images: {missingTest_images_count+imageIsFoundTest}\n")
                print(f"Total number of in test features : {len(test_features)}")
                print(f"Total number of test_label : {len(test_label)}")
                predict_label_SVM = SVM_Classifier.predict(test_features)
                predict_label_DT = DT_Classifier.predict(test_features)
                predict_label_KNN = KNN_Classifier.predict(test_features)
                print("\n **************** END test index ******************\n")

                confusionMatrixSVM = confusion_matrix(test_label, predict_label_SVM)
                confusionMatrixDT = confusion_matrix(test_label, predict_label_DT)
                confusionMatrixKNN = confusion_matrix(test_label, predict_label_KNN)
                print(f"\n I'm inside For loop of Confusion matrix is **ADDED ** \n")

                classification_report_svm.append(
                    classification_report(test_label, predict_label_SVM, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_dt.append(
                    classification_report(test_label, predict_label_DT, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_knn.append(
                    classification_report(test_label, predict_label_KNN, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))

                confusion_matrix_svm.append(confusionMatrixSVM)
                confusion_matrix_dt.append(confusionMatrixDT)
                confusion_matrix_knn.append(confusionMatrixKNN)
                print(f"\n -------> Confusion matrix is **APPENDED **\n")

                score_accuracy_SVM.append(accuracy_score(test_label, predict_label_SVM))
                score_accuracy_DT.append(accuracy_score(test_label, predict_label_DT))
                score_accuracy_KNN.append(accuracy_score(test_label, predict_label_KNN))
            SVM_RESULTS = [confusion_matrix_svm, classification_report_svm, score_accuracy_SVM]
            DT_RESULTS = [confusion_matrix_dt, classification_report_dt, score_accuracy_DT]
            KNN_RESULTS = [confusion_matrix_knn, classification_report_knn, score_accuracy_KNN]
            SVM.append(SVM_RESULTS)
            KNN.append(KNN_RESULTS)
            DT.append(DT_RESULTS)
        return SVM, KNN, DT

    def CrossValidation(self, image, labels, test_size):
        X = []
        Y = []
        X_train, X_test, y_train, y_test = train_test_split(image, labels, test_size=test_size)
        X.append(X_train)
        X.append(X_test)
        Y.append(y_train)
        Y.append(y_test)
        print()
        return X, Y

    def ROC_CURVE(self, label_test, label_score):
        n_classes = label_test.shape
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(label_test[:, i], label_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(label_test.ravel(), label_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # Plot of a ROC curve for a specific class
        plt.figure()
        plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr["micro"], tpr["micro"],
                 label='micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]))
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                           ''.format(i, roc_auc[i]))

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Some extension of Receiver operating characteristic to multi-class')
        plt.legend(loc="lower right")
        plt.show()


## **WorstCase**


In [ ]:
#not good
import errno
import os
from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm
from sklearn.metrics import roc_curve, auc, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


class Classification:
    def __init__(self, PATH_PROJECT):
        self.path_project = os.path.join(PATH_PROJECT, 'Classification')
        try:
            if not os.path.exists(self.path_project + 'SVM'):
                self.path_getColor = os.path.join(self.path_project, 'SVM')
                os.mkdir(self.path_getColor)
                print('SVM Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('SVM Directory Already Exists.')
            else:
                raise

    def readfeatures(self, features_Path):
        # featuresFile = open(features_Path, "r")
        # featuresData = csv.reader(featuresFile)
        featuresFile = pd.read_csv(features_Path, sep=',', header=None)
        names = featuresFile.iloc[:, 0]
        features = featuresFile.iloc[:, 1:]
        shapefile = featuresFile.shape
        col = []
        for x in range(0, shapefile[1]):
            if x == 0:
                col.append("NAME")
            else:
                col.append("VALOR-" + str(x))
        featuresFile.columns = col
        # print(featuresFile)
        return names, features

    def readLabels(self, labels_path):
        labels_path = os.path.join(labels_path, 'Labels.csv')
        labels = pd.read_csv(labels_path, sep=',', header=[0])
        return labels

    def classificatorSVM(self, features, labels):
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        clf = svm.SVC(gamma='scale')
        clf.fit(X, tags)
        return clf

    def DecisionTree(self, features, labels):
        dt = DecisionTreeClassifier(random_state=30, max_depth=300)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        dt.fit(X, tags)

        return dt

    def KNN(self, features, labels):
        knn = KNeighborsClassifier(n_neighbors=200, algorithm='auto', weights='distance', n_jobs=-1)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        knn.fit(X, tags)
        return knn

    def classification(self, path_dataset, features, labels, n_splits, tags, target_names, vals_to_replace):
        pd.options.mode.chained_assignment = None
        onlyfiles = [f for f in listdir(path_dataset) if
                     isfile(join(path_dataset, f))]
        #print(onlyfiles)
        k_folds = KFold(n_splits=n_splits)
        SVM = []
        KNN = []
        DT = []
        for stage, feature in zip(labels, features):
            stage['Color'] = stage['Color'].map(vals_to_replace)
            labels_color = stage['Color'].to_numpy().tolist()
            images_name = stage['Nombre de la imagen'].to_numpy().tolist()
            svm_training_Score = []
            confusion_matrix_svm = []
            confusion_matrix_dt = []
            confusion_matrix_knn = []
            classification_report_svm = []
            classification_report_dt = []
            classification_report_knn = []
            score_accuracy_SVM = []
            score_accuracy_DT = []
            score_accuracy_KNN = []
            index_images_name = []
            k_folds.get_n_splits(index_images_name)

            for train_index, test_index in k_folds.split(images_name):
                train_label = []
                test_label = []
                train_features = []
                test_features = []
                for i in train_index:
                  try:
                      train_features.append(feature.to_numpy().tolist()[images_name.index(onlyfiles[i].split('.')[0])])
                      train_label.append(labels_color[images_name.index(onlyfiles[i].split('.')[0])])
                  except ValueError:
                      print(f"Warning: '{onlyfiles[i].split('.')[0]}' not found in images_name. Skipping...")
                continue
                    #train_features.append(feature.to_numpy().tolist()[images_name.index(onlyfiles[i].split('.')[0])])
                    # train_features.append(feature.to_numpy()[images_name.str(onlyfiles[i].split('.')[0])])
                    #train_label.append(labels_color[images_name.index(onlyfiles[i].split('.')[0])])
                SVM_Classifier = self.classificatorSVM(train_features, train_label)
                DT_Classifier = self.DecisionTree(train_features, train_label)
                KNN_Classifier = self.KNN(train_features, train_label)

                for i in test_index:
                    test_features.append(feature.to_numpy()[images_name.index(str(onlyfiles[i].split('.')[0]))])
                    test_label.append(labels_color[images_name.index(str(onlyfiles[i].split('.')[0]))])
                predict_label_SVM = SVM_Classifier.predict(test_features)
                predict_label_DT = DT_Classifier.predict(test_features)
                predict_label_KNN = KNN_Classifier.predict(test_features)

                confusionMatrixSVM = confusion_matrix(test_label, predict_label_SVM)
                confusionMatrixDT = confusion_matrix(test_label, predict_label_DT)
                confusionMatrixKNN = confusion_matrix(test_label, predict_label_KNN)

                classification_report_svm.append(
                    classification_report(test_label, predict_label_SVM, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_dt.append(
                    classification_report(test_label, predict_label_DT, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_knn.append(
                    classification_report(test_label, predict_label_KNN, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))

                confusion_matrix_svm.append(confusionMatrixSVM)
                confusion_matrix_dt.append(confusionMatrixDT)
                confusion_matrix_knn.append(confusionMatrixKNN)

                score_accuracy_SVM.append(accuracy_score(test_label, predict_label_SVM))
                score_accuracy_DT.append(accuracy_score(test_label, predict_label_DT))
                score_accuracy_KNN.append(accuracy_score(test_label, predict_label_KNN))
            SVM_RESULTS = [confusion_matrix_svm, classification_report_svm, score_accuracy_SVM]
            DT_RESULTS = [confusion_matrix_dt, classification_report_dt, score_accuracy_DT]
            KNN_RESULTS = [confusion_matrix_knn, classification_report_knn, score_accuracy_KNN]
            SVM.append(SVM_RESULTS)
            KNN.append(KNN_RESULTS)
            DT.append(DT_RESULTS)
        return SVM, KNN, DT

    def CrossValidation(self, image, labels, test_size):
        X = []
        Y = []
        X_train, X_test, y_train, y_test = train_test_split(image, labels, test_size=test_size)
        X.append(X_train)
        X.append(X_test)
        Y.append(y_train)
        Y.append(y_test)
        print()
        return X, Y

    def ROC_CURVE(self, label_test, label_score):
        n_classes = label_test.shape
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(label_test[:, i], label_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(label_test.ravel(), label_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # Plot of a ROC curve for a specific class
        plt.figure()
        plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr["micro"], tpr["micro"],
                 label='micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]))
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                           ''.format(i, roc_auc[i]))

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Some extension of Receiver operating characteristic to multi-class')
        plt.legend(loc="lower right")
        plt.show()

In [ ]:
#not good

import errno
import os
import sys

import cv2 as cv
import numpy as np
from tqdm import tqdm

import Source.Classification as Cl
import Source.FeatureExtraction as fE
import Source.PreProcessingData as pD
import Source.ReadImages as rI


def show(image):
    cv.imshow('Imagen ', image)
    cv.waitKey(0)
    cv.destroyAllWindows()


class MainClass:
    PROJECT_PATH = os.path.join(os.getcwd(), os.path.pardir)
    PATH_IMAGES_ORIGINAL = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "DATASET - Original"))
    PATH_Labels = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "Labels"))
    PATH_LabelsXML = os.path.abspath(os.path.join(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "Labels"), "LabelsXML"))
    PATH_IMAGES = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "DATASET"))
    readimages = None
    preprocessing = None
    featureExtraction = None
    clasification = None

    def __init__(self):
        try:
            if not os.path.exists(self.PROJECT_PATH + 'PreProcessing'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'PreProcessing'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('PreProcessing Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'FeatureExtraction'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'FeatureExtraction'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('FeatureExtraction Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'Sampling'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'Sampling'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('Sampling Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'Classification'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'Classification'))
                print('Directory Classification Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('Classification Directory Already Exists.')
            else:
                raise
        self.readimages = rI.LoadData(self.PATH_IMAGES_ORIGINAL)
        self.preProcessing = pD.PreProcessingData(self.PROJECT_PATH, self.PATH_IMAGES_ORIGINAL)
        self.featureExtraction = fE.FeatureExtraction(self.PROJECT_PATH, self.PATH_IMAGES_ORIGINAL)
        self.clasification = Cl.Classification(self.PROJECT_PATH)

    def main_run(self):
        # Se declaran las clases para poder utilizar los elementos
        read = self.readimages
        pp = self.preProcessing
        fe = self.featureExtraction

        # Se lee el nombre y la imagen que se encuentre en el PATH del dataset ORIGINAL
        # img, name = read.read_One_Image(self.PATH_IMAGES)
        # Se lee el nombre y la imagen que se encuentre en el PATH del dataset RECORTADO
        img, name = read.read_One_Image(self.PATH_IMAGES)

        # Se obtiene las dimensiones de la imagen original
        height_ori, width_ori, depth_ori = img.shape
        # print("Image original shape: \n Height:", height_ori, ", Width:", width_ori)

        # Se realiza un ajuste de tamaño para reducir la imagen a unas dimensiones de 600x400
        img_resize = pp.resize_Image(img, name)

        # La imagen reajustada se convierte de BGR a RGB
        img_resize = cv.cvtColor(img_resize, cv.COLOR_BGR2RGB)
        # Se convierte la imagen de RGB a HSV
        hsv_image = pp.rgb_2_HSV(img_resize, name)
        # Se saca la imagen en pila de tono de rojos
        stack, name = pp.stackColors(hsv_image, name)
        # Se saca los histogramas por imagen para determinar el rango de color de los dientes
        pp.hsv_hist(hsv_image, name)
        #
        # plt.imshow(img_resize)
        # Blur image slightly
        name, blurimage = pp.blurImage(img_resize, name)
        pp.show_mask(blurimage, name)
        pp.overlay_mask(blurimage, img_resize, name)

        # Se obtiene la rueda cromatica de la imágen
        # pp.getChromatiColor(img_resize,name,fe)

        # img_rgb2ycbcr = pp.rgb_2_YCrCb(img_resize, name)
        img_rgb2hsv = pp.rgb_2_HSV(img_resize, name)

    '''easygui.msgbox("Image original shape: \n Height:" + str(height_ori) + "px, Width:" + str(width_ori) + "px" +
                   "\n Image Resize shape: \n Height:" + str(height_res) + "px, Width:" + str(width_res) + "px",
                   image=os.path.join(os.path.join(os.getcwd(), os.path.pardir),
                                      'PreProcessing/ResizeImages/' + name),
                   title="Image Shape - PreProcessing ")'''

    def savebin(self):
        read = self.readimages
        pp = self.preProcessing
        fe = self.featureExtraction
        images, names = read.read_Images(self.PATH_IMAGES_P)
        for image_point, name_point in zip(images, names):
            img_resize = pp.resize_Image(image_point, name_point)
            img_resize = cv.cvtColor(img_resize, cv.COLOR_BGR2RGB)
            img_resize = cv.cvtColor(img_resize, cv.COLOR_RGB2GRAY)
            pp.bin(img_resize, name_point)

    def main_alldataset(self):
        # Se declaran las clases para poder utilizar los elementos
        read = self.readimages
        pp = self.preProcessing
        fe = self.featureExtraction
        cc = self.clasification
        doption = input(
            'Que desea hacer? \n1. Leer imagenes y obtener caracteristicas'
            '\n2. Leer archivo de caracteristicas y entrenar algoritmo\n')
        option = int(doption)
        print(option)

        if option == 1:
            images, names = read.read_Images(self.PATH_IMAGES)
            bar = tqdm(images, ncols=len(images), unit=' image')
            for image_point, name_point in zip(bar, names):
                bar.set_description("Procesando imagen %s" % name_point)
                # Se reajusta la imagen a un tamaño de 600x400px
                img_resize = pp.resize_Image(image_point, name_point)

                # La imagen reajustada se convierte de BGR a RGB
                img_resize = cv.cvtColor(img_resize, cv.COLOR_BGR2RGB)

                # Se convierte la imagen de RGB a HSV
                hsv_image = pp.rgb_2_HSV(img_resize, name_point)

                # Se saca la imagen en pila de tono de rojos
                stack, name_point = pp.stackColors(hsv_image, name_point)

                # Se saca los histogramas por imagen para determinar el rango de color de los dientes
                pp.hsv_hist(hsv_image, name_point)

                # Blur image slightly
                name_point, blurimage = pp.blurImage(img_resize, name_point)
                '''file_ = open(os.path.join(self.PROJECT_PATH, 'Pruebas') + name_point + '.txt', "w")
                for i in blurimage:
                    file_.write(str(i))
                file_.close()'''

                # A partir del rango de color, se saca una máscara donde se ubican los dientes y se procede a buscar el contorno más grande dentro del área objetivo.
                mask = pp.findBiggestContour(blurimage, name_point)
                # Se separan los canales de la
                # imágen en RGB
                channelR, channelG, channelB = cv.split(img_resize)
                # Se obtienen los momentos de color de cada espacio de color
                red = fe.getFeaturesVector(channelR, mask)
                green = fe.getFeaturesVector(channelG, mask)
                blue = fe.getFeaturesVector(channelB, mask)
                # colores = ["RED", "GREEN", "BLUE"]

                # Se obtiene una mascara de la imágen compuesta por los valores de los canales donde se encontraron los datos de interes.
                imagen = [red, green, blue]
                # Se coloca la ruta del archivo de caracteristicas
                filefeaturespath = os.path.join(os.path.join(self.PROJECT_PATH, 'FeatureExtraction'), 'features.csv')
                # Se escribe en un archivo las caracteristicas del dataset
                fe.getFeatures(imagen, filefeaturespath, name_point)
                fileLabels = open(os.path.join(self.PATH_Labels, 'Labels.csv'))
                pp.show_mask(blurimage, name_point)
                pp.overlay_mask(blurimage, img_resize, name_point)
        elif option == 2:
            times_execution = -1
            while times_execution <= 0:
                times_execution = int(input('\nIndique la cantidad de veces de ejecución:'))
            bar = tqdm(range(times_execution), unit=' times')
            for i in bar:
                # Se lee el archivo de caracteristicas donde se encuentran los momentos de color
                filefeaturespath = os.path.join(os.path.join(self.PROJECT_PATH, 'FeatureExtraction'), 'features.csv')
                names, features = cc.readfeatures(filefeaturespath)
                labels = cc.readLabels(self.PATH_Labels)
                features_images = features.values
                vals_to_replace = {'a1': '0', 'a2': '1', 'a3': '2', 'a35': '3'}
                tags = ['0', '1', '2', '3']
                target_names = ['a1', 'a2', 'a3', 'a35']
                folds = int(input('\nCantidad de folios a seperarar el conjunto de datos:'))
                test_size = int(input('\n Porcentaje de division del conjunto de datos trianing/test:'))
                X, Y = cc.CrossValidation(features, labels, test_size / 100)
                SVM, DT, KNN = cc.classification(self.PATH_IMAGES, X, Y, folds, tags, target_names, vals_to_replace)
                print('--------- Training ---------')
                for S, D, K in zip(SVM, DT, KNN):

                    matrix_confusion_SVM, report_clasification_SVM, report_scores_SVM = S.split()
                    matrix_confusion_DT, report_clasification_DT, report_scores_DT = D.split()
                    matrix_confusion_KNN, report_clasification_KNN, report_scores_KNN = K.split()
                    print('\n')
                    print('-------SVM------')
                    for report in report_clasification_SVM:
                        print(report)
                        for item in report:
                            print(report[item])
                    print('--------MEAN SVM--------')
                    print(np.mean(report_scores_SVM))
                    print('-------DT------')
                    for report in report_clasification_DT:
                        print(report)
                        for item in report:
                            print(report[item])
                    print('--------MEAN DT--------')
                    print(np.mean(report_scores_DT))

                    print('-------KNN------')
                    for report in report_clasification_KNN:
                        print(report)
                        for item in report:
                            print(report[item])
                    print('--------MEAN KNN--------')
                    print(np.mean(report_scores_KNN))
                    print('--------- TEST ---------')


if __name__ == '__main__':
    tesis: MainClass = MainClass()
    # tesis.main_run()
    tesis.main_alldataset()
    # tesis.savebin()
    print('Se ha finalizado la ejecución del experimento')
    sys.exit(0)
